# Data cleaning script 

## Importing required libraries

In [121]:
import pandas as pd
import string
import re
import requests
import nltk
# Uncomment if not downloaded to your device
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('wordnet')

### Upload csv file
We are using data from 2 sources; Nexus and Kaggle. The Nexus data is already fairly clean as it allowed us to choose which columns we required and filter the data before downloading it to a csv. 

The Kaggle data requires a bit more cleaning and reorganising before we are able to concatonate the data sets together. 

In [122]:
#Upload Nexus csv 
Nexus_csv_path = 'Headline_data_v2.csv'
Nexus_data_raw = pd.read_csv(Nexus_csv_path, encoding='windows-1252')

#Upload Kaggle csv
Kaggle_csv_path = 'Kaggle data.csv'
Kaggle_data_raw = pd.read_csv(Kaggle_csv_path, encoding='utf-8')

## Restructuring Kaggle data 
The Kaggle data has extra information on the price of Bitcoin. This information is not given in the Nexus data set and to keep the consistency it is better to get the Bitcoin price information from the same source to cover teh entire data set. This means these collumns can be dropped. The symbol column can also be dropped the whole column is 'BTC' as an entry which doesn't provide much insite. 

The kaggle data has multiple headlines per date. This needs to be restructured to 1 headline per row.

The collumn also need to be renamed to be consistent with the Nexus data. 

In [123]:
# Drop unnecessary columns 
Kaggle_data = Kaggle_data_raw.drop(columns=['Unnamed: 0', 'open_price', 'close_price','high_price', 'low_price', 'symbol'])

In [124]:
# Separates each headline onto a new row.
Kaggle_data['articles'] = Kaggle_data['articles'].apply(lambda x: x.split("', "))
Kaggle_data = Kaggle_data.explode('articles').reset_index(drop=True)

In [125]:
# Rename collumns in order to combine with Nexus data
Kaggle_data = Kaggle_data.rename(columns={'begins_at': 'Published date', 'articles': 'Headline'})

## Removing non-English headlines
When downloading the Nexus data we were able to pick parameters for the data we wanted, this meant we could select only English language papers. For the Kaggle data there is a mixture of different languages. As we are only focusing on English language papers these need to be removed. Dectecting language takes a lot of computing power so it is important to do this step before combining with the Nexus data. 

To remove them, a new column is created which has the detected language in it. If the language is not detected it is set to 'unknown', this prevents error messages. Then only the 'en' language papers are kept in the data frame and the lang collumn is dropped as it is no longer needed. 

Originally when running the code there was a SettingWithCopyWarning. To avoid this warning .loc has been used to access the lang column. For further information on SettingWithCopyWarning 'https://www.analyticsvidhya.com/blog/2021/11/3-ways-to-deal-with-settingwithcopywarning-in-pandas/#:~:text='SettingWithCopy'%20is%20a%20common%20warning,settingwithcopywarning%20new%20column%20pops%20up'.

In [126]:
#Dectect language of newspaper 
from langdetect import detect

# Safe detect for when the language is not detecable
def safe_detect(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'

# This function takes a long time to perform
# Use .loc to set the new column to avoid SettingWithCopyWarning
Kaggle_data.loc[:, 'lang'] = Kaggle_data['Headline'].apply(lambda x: safe_detect(x))
Kaggle_data.head()


,Published date,Headline,lang
0,25/02/2018,['Original Pizza Day Purchaser Does It Again W...,en
1,25/02/2018,'This 11-year-old just wrote a book on bitcoin...,en
2,25/02/2018,"'Without Mentioning Blockchain, Putin Says Tha...",en
3,25/02/2018,'El comprador original del Pizza Day lo hace d...,es
4,25/02/2018,'Meet the strippers tattooed with BARCODES so ...,en


In [127]:
# Remove non-English headlines
Kaggle_data = Kaggle_data[Kaggle_data['lang'] == 'en']
Kaggle_data = Kaggle_data.drop('lang', axis='columns')

## Remove headlines with CRYPTO[...]World Markets
There are a lot of news headlines in the nexus data with this structure and they do not provide any extra information so it is best ot remove them. 

In [128]:
pattern = r'CRYPTO.*World Markets'

# Filter out rows containing the specific pattern
Nexus_data = Nexus_data_raw[~Nexus_data_raw['Headline'].str.contains(pattern, regex=True, na=False)]

## Combining data into a single data frame.
The Kaggle data has been reshaped and renamed so that is is now compatible to combine with the Nexus data. The Kaggle data only contains the publish date and the headline, whereas the nexus data has 2 other columns. This will just mean that for Kaggle data there will be an NA for entries in these columns. 

In [129]:
headline_data = pd.concat([Nexus_data_raw, Kaggle_data])

#Checks
print(len(Nexus_data_raw) + len(Kaggle_data))
print(len(headline_data))

19364
19364


## Cleaning data frame.
The duplicate rows are removed as this could alter the senitment analysis if there are multiple entries for one headline. The null headlines are also removevd as they provide no extra information. 

The data then needs to be cleaned specifically for sentiment analysis. Headlings are lower cased and unctuation removed to reduce noise and so that the model can focus on the more meaningful parts of the headline. 

#### Remove duplicate rows

In [130]:
# Remove dulpicate rows
headline_data.drop_duplicates(subset=['Headline'], keep="first", inplace=True)

#### Remove Null headlines

In [132]:
# Removes all missing headlines
headline_data.dropna(subset=['Headline'])
headline_data['Headline'] = headline_data['Headline'].fillna('')

,Published date,Countries,Company,Headline
0,30/05/2024,England & Wales,NaN,Woman jailed for Bitcoin laundering
1,06/06/2024,England & Wales,BEST INC,Best Crypto Casino Sites & Bitcoin Casinos in ...
2,21/05/2024,United Kingdom of Great Britain and Northern I...,NaN,Wright lied extensively as 'bitcoin inventor'
3,26/05/2024,United Kingdom of Great Britain and Northern I...,METROPOLITAN BANK HOLDING CORP,Ex-takeaway worker with Bitcoin worth more tha...
4,21/05/2024,United Kingdom of Great Britain and Northern I...,BIRD & BIRD LLP,Inventor of bitcoin' given court rebuke


#### Lower case headlines

In [133]:
# Lower case headlines
# Loops through every word in the the Title collumn of the data frame and applies the lower() function
# The lower function only works on strings, so the Title collumn must be split up first and then rejoined again after. 
headline_data['Cleaned Headline'] = headline_data['Headline'].apply(lambda x: ' '.join(word.lower() for word in x.split()))

#### Remove punctuation
We don't want to remove punctuation entirely as Vader and TextBlob can process and use punction to help. This is a little clean up to standardise the punctuation in the head line. I have added a space between any punctuation and words, this is because to the program punctuation changed the string e.g. if we wanted to remove the stop word if, 'if' and 'if,' are not read as equavalient strings and so the word isn't removed. 

In [134]:
# Remove punctuation

# Strip \n 
# This symbol indicates a new line. As it involves a punctuation symbol, this step must be done before removing punctuation.
headline_data['Cleaned Headline'] = headline_data['Cleaned Headline'].str.replace(r'\n', '', regex=True)

# Remove anything after the '|'. This si because it is jsut the name of the newspaper and not the headline.
headline_data['Cleaned Headline'] = headline_data['Cleaned Headline'].str.split('|').str[0]

#This separates the words from punctuation this is important to stop 
#'bitcoin bitcoin, and bitcoin being counted as different words. 
def add_spaces_around_punctuation(text):
    # Define a regex pattern for punctuation
    pattern = r'([.,!?;:()\'"\[\]{}<>@#$%^&*+=~`|\\/-])'
    # Use re.sub to add spaces around each punctuation mark
    spaced_text = re.sub(pattern, r' \1 ', text)
    # Remove any extra spaces that might appear
    spaced_text = re.sub(r'\s+', ' ', spaced_text).strip()
    return spaced_text

headline_data['Cleaned Headline'] = headline_data['Cleaned Headline'].apply(add_spaces_around_punctuation)

### Remove stop words
Stop words are a set of commonly used words in a language. In the context of NLP, they carry very little meaning and so are removed to further reduce noise. 

The library of stop words from the nltk module is used to initially remove stop words. Then there is a count of most commonly occuring words in the data set. From there more specific to crypto stop words can be picked out and removed. 

In [135]:
# Remove stop words

# Imported list of English stop words for NLTK
stop_words = nltk.corpus.stopwords.words('english')

# Loops through and if word not in the stop words list then it is kept in the headline. (all words in stop word list removed)
headline_data['Cleaned Headline'] = headline_data['Cleaned Headline'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

### Dectecting more specific stop words

In [139]:
# Create our own list of stop words. The imported stop words are a good start but not specific to crypto headline data.

# Check frequency of words.

# Combine all headlines into a single string and split into words
all_words = " ".join(headline_data['Cleaned Headline']).split()

# Create a pandas Series from the words
words_series = pd.Series(all_words)

# Filter out non-alphabetic words and count occurrences
word_counts = words_series[words_series.str.isalpha()].value_counts()[:30]
print(word_counts)

bitcoin           12519
btc                2712
crypto             2264
price              2004
new                 783
ethereum            747
says                709
market              686
cryptocurrency      683
mining              644
analysis            542
could               469
us                  425
million             421
time                395
high                381
first               378
year                365
eth                 354
trading             343
blockchain          337
buy                 322
exchange            320
money               302
world               295
gold                291
markets             285
top                 282
investors           278
xrp                 263
Name: count, dtype: int64


### Selecting specific stop words to remove

In [140]:
# This step is very subjective. 
# It is just deciding which words aren't as relavent to help reduce noise in the sentiment analysis
manual_stop_words = ['us', 'says', 'sam', 'sec', 'could', 'bankmanfried', 'man', 'wall', 'woman', 'briefing', 'amid', 'lucy', 'know']
headline_data['Cleaned Headline'] = headline_data['Cleaned Headline'].apply(lambda x: ' '.join(word for word in x.split() if word not in manual_stop_words))

### Changing abbreviations

In [141]:
# Looking at the word freq I noticed 5bn. I am going to separate numbers and replace bn with billion and mn with million.
headline_data['Cleaned Headline'] = headline_data['Cleaned Headline'].str.replace(r"bn", r" billion" , regex=True).replace(r"mn", r" million" , regex=True)

## Headline relevance checker
Due to the sheer number of headlines it is too time consuming to go through every one to decide if they are relavent to bitcoin or not. To get around this a new column is added to the data frame that has the number of crypto relavent words in the headline. 

There were no suitable pre made list of crypto relavent words online. This mean we have to scrap the website 'https://cryptonest.co.uk/pages/crypto-dictionary' for all their crypto definition using beautiful soup. All the definition words were within the class 'page-content page-content--medium rte' and written in bold so it was easy to locate and isolate only the key words. 

Challenges arose as some key words had their corresponing abbreviation in brackets e.g Ether (ETH). Some keywords had a slash in them to indicate similar words e.g. bear / bearish. This meant that the string to compare to the headline was with key word and abbreviaiton. These needed to be separated out into two different key words so that they key word and abbreviaiton could be compared separately. 

A set was used to make sure there were no duplicate words in the keywords. This was then converted back into a list as sets are immutable. 

#### Scraping website

In [142]:
from bs4 import BeautifulSoup

# Website with crypto dictionary
url = 'https://cryptonest.co.uk/pages/crypto-dictionary' 

response = requests.get(url)
html_content = response.text

# Parse the content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Class containing all the definitions in
class_name = "page-content page-content--medium rte" 

# Find all elements with the specified class
elements_with_class = soup.find_all(class_=class_name)


#### Cleaning Key words to be compatible with headlines.

In [143]:
# Define a regular expression pattern to remove all punctuation except brackets

# Initialize a set to store cleaned words
crypto_words_set = set()

# Regular expression to find text within brackets
brackets_pattern = re.compile(r'\(([^)]+)\)')

# Iterate over elements and find all <strong> tags within each
for element in elements_with_class:
    strong_tags = element.find_all('strong')
    for key_words in strong_tags:
        # Get the text and convert to lowercase
        text = key_words.get_text().lower()
        
        # Remove hyphens 
        text = text.replace('-', ' ')
        
        # Extract bracketed words
        bracketed_words = brackets_pattern.findall(text)
        
        # Remove bracketed words from the original text
        text_without_brackets = re.sub(brackets_pattern, '', text)
        
        # Process original text without bracketed parts
        split_words = text_without_brackets.split('/')
        
        # Add non-bracketed words to the set
        for word in split_words:
            cleaned_word = word.strip()
            if cleaned_word:
                crypto_words_set.add(cleaned_word)
        
        # Add bracketed words separately to the set
        for word in bracketed_words:
            cleaned_word = word.strip()
            if cleaned_word:
                crypto_words_set.add(cleaned_word)

#### Adding missed words.

In [144]:
# From scanning the list crypto and bitcoins werent included so manually adding it to the set.
# Bitcoins doesn't lemmatise to bitcoin I tihnk this is because it is specific to crypto.
crypto_words_set.add('bitcoins')

# Convert to a list so it can be used in function below
crypto_words = list(crypto_words_set)


#### Adding count collumn for number of crypto relavent word

In [145]:
# Some words contain crypto e.g. cryptoqueen were not being picked up. This ensures all variations are counted
containing_crypto = re.compile(r'\b\w*crypto\w*\b')
containing_coin = re.compile(r'\b\w*coin\w*\b')

# Apply the function to calculate the count
headline_data['Count'] = headline_data['Cleaned Headline'].apply(
    lambda x: sum(1 for word in x.split() if containing_crypto.search(word) or containing_coin.search(word) or word in crypto_words)
)

#### Selecting only the relavent headlines.
A csv is made of both relavent and non-relavent so they can be skimmed through to double check if the are relavent or not. 

In [146]:
# Create a csv of all the non-relavent headlines which can be double checked to make sure they are not relavent. 
countdf = headline_data[headline_data['Count'] == 0]
countdf.to_csv('Count_0.csv')

# Create a csv of all the relavent headlines to double check they are relavent. 
crypto_headline_data = headline_data[headline_data['Count'] != 0]



### Lemmatising words
Lemmatising is the process to reverting words back to their root. This process is requires a lot of computing power so best to do it at the end when there is only the relavent headlines in their cleanest form. 

In [147]:
# Import textblob
from textblob import Word

# Lemmatise final review format
crypto_headline_data.loc[:, 'Lem Headline'] = crypto_headline_data['Cleaned Headline']\
.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

crypto_headline_data.to_csv('crypto_headline_data.csv', index=False)



C:\Users\josie\AppData\Local\Temp\ipykernel_19676\134444803.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crypto_headline_data.loc[:, 'Lem Headline'] = crypto_headline_data['Cleaned Headline']\
